#### Importing the features required for prediction of patent class

In [2]:
import pandas as pd
features = pd.read_csv("./data/patent_class_features_after_2022_20230611.csv")
class_labels = pd.read_csv("./data/patent_class_after_2022_20230611.csv")

In [3]:
features.shape

(344328, 9)

In [4]:
features.columns.dtype

dtype('O')

In [5]:
class_labels.shape

(461258, 2)

In [6]:
features.head()

,PATENT,PUB_DATE,COUNTRY,PRIMARY_LANG,IPC,DWPI_TITLE,DWPI_DETAILED_DESCRIPTIONS,DWPI_NOVELTY,DWPI_USE
0,CN218280954U_,2023-01-13,CN,ZH_ZH,"['B01D004604', 'B01D004676', 'B01D004602', 'B0...",Surface anti-sticking dust-removing filter bag...,NaN,The utility model claims a surface anti-sticki...,NaN
1,CN218188237U_,2023-01-03,CN,ZH_ZH,"['B01D002996', 'B01D002933']",Environment-friendly reusable cooling liquid f...,NaN,The utility model claims a new environment-fri...,NaN
2,CN218442764U_,2023-02-03,CN,ZH_ZH,"['F24H000118', 'B08B000304', 'F24H000902', 'F2...",Organic silicon modified epoxy heat conducting...,NaN,The utility model claims an organic silicon mo...,NaN
3,CN218428090U_,2023-02-03,CN,ZH_ZH,['B25B002700'],"Electronic screen dismounting device, has lowe...",An INDEPENDENT CLAIM is included for the elect...,The utility model relates to screen disassembl...,NaN
4,CN218195410U_,2023-01-03,CN,ZH_ZH,"['B26D000702', 'B26D000510', 'B26D000108']","Inclined cutting paper cutting device, has cut...",NaN,The utility model claims an inclined cutting p...,NaN


In [7]:
# PATENT, DWPI_TITLE and DWPI_NOVELTY are object data type, have to convert them to string
features['PATENT'] = features['PATENT'].astype(str).str.strip()
class_labels['PATENT'] = class_labels['PATENT'].astype(str).str.strip()
features['DWPI_TITLE'] = features['DWPI_TITLE'].astype(str).str.strip()
features['DWPI_NOVELTY'] = features['DWPI_NOVELTY'].astype(str).str.strip()

# Only first 2 characters of PRIMARY_LANG before delimeter "_" is required
def get_left_part(s):
    return s.split('_')[0]
features.PRIMARY_LANG = features.PRIMARY_LANG.apply(get_left_part)

# PUB_DATE column has to be parsed as pandas datetime object
features['PUB_DATE'] = pd.to_datetime(features['PUB_DATE'], format='%Y-%m-%d')
# Calculate the date range
start_date = features['PUB_DATE'].min()
end_date = features['PUB_DATE'].max()
# Print the date range
print(f"Date range: {start_date.date()} to {end_date.date()}")

Date range: 2023-01-01 to 2023-02-28


In [8]:
features.nunique()

PATENT                        344328
PUB_DATE                          59
COUNTRY                           40
PRIMARY_LANG                      23
IPC                           264953
DWPI_TITLE                    343353
DWPI_DETAILED_DESCRIPTIONS     85860
DWPI_NOVELTY                  343944
DWPI_USE                      147691
dtype: int64

In [9]:
missing_values_count = features.isna().sum()
print(missing_values_count)

PATENT                             0
PUB_DATE                           0
COUNTRY                            0
PRIMARY_LANG                       0
IPC                               15
DWPI_TITLE                         0
DWPI_DETAILED_DESCRIPTIONS    251364
DWPI_NOVELTY                       0
DWPI_USE                      193017
dtype: int64


In [10]:
# representing missing values as percentages (proportion)
# note IPC column still has 15 missing values
missing_values_count = round((features.isna().sum()/features.shape[0])*100,4)
print(missing_values_count)

PATENT                         0.0000
PUB_DATE                       0.0000
COUNTRY                        0.0000
PRIMARY_LANG                   0.0000
IPC                            0.0044
DWPI_TITLE                     0.0000
DWPI_DETAILED_DESCRIPTIONS    73.0013
DWPI_NOVELTY                   0.0000
DWPI_USE                      56.0561
dtype: float64


In [11]:
# Count duplicates for each column
duplicate_counts = {col: features[col].duplicated().sum() for col in features.columns}

# Display results in terms of %
for col, count in duplicate_counts.items():
    print(f"Column '{col}' has {round(count/features.shape[0]*100,2)}% duplicate values.")

Column 'PATENT' has 0.0% duplicate values.
Column 'PUB_DATE' has 99.98% duplicate values.
Column 'COUNTRY' has 99.99% duplicate values.
Column 'PRIMARY_LANG' has 99.99% duplicate values.
Column 'IPC' has 23.05% duplicate values.
Column 'DWPI_TITLE' has 0.28% duplicate values.
Column 'DWPI_DETAILED_DESCRIPTIONS' has 75.06% duplicate values.
Column 'DWPI_NOVELTY' has 0.11% duplicate values.
Column 'DWPI_USE' has 57.11% duplicate values.


**It means PATENT, IPC, DWPI_TITLE, DWPI_NOVELTY are significanlty important to differentiate between patents and maybe for even classification purposes.**

**Whereas DWPI_DETAILED_DESCRIPTIONS and DWPI_USE with a lot of duplicate and missing values seem insignificant. Dropping them makes sense.**

**Columns "PUB_DATE", "COUNTRY" and "PRIMARY_LANG" also seem insignificant for differentiating between patents and their respective classes.**

In [12]:
# Dropping insignificant columns
features = features.drop(columns=['DWPI_DETAILED_DESCRIPTIONS', 'DWPI_USE'])

In [13]:
class_labels.nunique()

PATENT    344328
CLASS        287
dtype: int64

#### There are 287 unique classes for 344328 unique patents. But the class_labels DataFrame has 344328 unique patents with 461258 rows which means that some patents belong to multiple classes

In [14]:
class_labels = class_labels.groupby('PATENT')['CLASS'].apply(list).reset_index()
class_labels

,PATENT,CLASS
0,AR123668A1,[Q31]
1,AR123669A1,"[T01, P14]"
2,AR123694A1,[Q49]
3,AR123695A1,"[S03, J04]"
4,AR123703A1,[A93]
...,...,...
344323,ZA202213920B_,[P14]
344324,ZA202300109B_,[Q43]
344325,ZA202300118B_,"[S05, T01]"
344326,ZA202300175B_,"[A96, D22, F07, P32]"


In [15]:
class_labels.shape

(344328, 2)

In [16]:
unmatched_in_features = features[~features['PATENT'].isin(class_labels['PATENT'])]
unmatched_in_class_labels = class_labels[~class_labels['PATENT'].isin(features['PATENT'])]

print("Patents in features but not in class_labels:", len(unmatched_in_features))
print("Patents in class_labels but not in features:", len(unmatched_in_class_labels))

Patents in features but not in class_labels: 0
Patents in class_labels but not in features: 0


In [17]:
#### There are 344328 unique patents in both the data frames with exact match so doing a inner join
df = features.merge(class_labels, on='PATENT', how='inner')
df.shape

(344328, 8)

In [18]:
# Which columns have missing values after the join ?
missing_values_count = df.isna().sum()
print(missing_values_count)

PATENT           0
PUB_DATE         0
COUNTRY          0
PRIMARY_LANG     0
IPC             15
DWPI_TITLE       0
DWPI_NOVELTY     0
CLASS            0
dtype: int64


In [19]:
# Dropping rows where IPC values are missing. It wouldn't hurt because these 15 rows account for only 0.0044% of entire dataset.
df = df.dropna(subset=['IPC'])
missing_values_count = df.isna().sum()
print(missing_values_count)
# Finally, no missing values. The dataset is still big enough for models to be trained upon. 

PATENT          0
PUB_DATE        0
COUNTRY         0
PRIMARY_LANG    0
IPC             0
DWPI_TITLE      0
DWPI_NOVELTY    0
CLASS           0
dtype: int64


In [20]:
# Just looking at some examples where Class has multiple values
df_multi_class = df[df['CLASS'].apply(lambda x: len(x) > 1)]
df_multi_class.head()

,PATENT,PUB_DATE,COUNTRY,PRIMARY_LANG,IPC,DWPI_TITLE,DWPI_NOVELTY,CLASS
52,CN218439434U_,2023-02-03,CN,ZH,"['E21F000500', 'B03C0003017']",Automatic cleaning screen for tunnel air purif...,The utility model claims an automatic cleaning...,"[J01, X25]"
240,FR3124913A1,2023-01-06,FR,FR,"['H04L00410816', 'H04L001200', 'H04L004500', '...",Method for managing orchestration entity in so...,The method involves obtaining (E70) an indicat...,"[T01, W01]"
241,CN218227814U_,2023-01-06,CN,ZH,"['B29C004956', 'B29C004942', 'B29C004964', 'B2...",nan,The machine has a pull plate device whose bott...,"[A92, A32]"
245,WO2023279108A1,2023-01-05,WO,EN,"['C12N001586', 'A61K004800', 'C07K001447']",New nucleic acid molecule comprising an adeno-...,"Nucleic acid molecule, comprising an adeno-ass...","[B04, D16]"
247,CN115584655A_,2023-01-10,CN,ZH,"['C05B000700', 'D21B000116', 'D21C000104', 'D2...",Utilizing wheat straw crushed by-product bio-f...,Utilizing wheat straw crushed by-product bio-f...,"[F09, C04]"
